# Baseball Play Description Inference Model - Run Calculation

This notebook runs code to train an LLM to take a set of prompts with play descriptions from a baseball game and return the number of runs a specific team scored during the game.

## Step 0: Load Necessary Python Modules

The modules **datasets**, **accelerate**, and **peft** are not included in the Google Colab environment, but they are required to run the Trainer object. Upon opening this notebook or connecting to a new runtime, run this cell and restart the runtime to let the changes take effect.

In [ ]:
!pip install datasets
!pip install accelerate
!pip install peft

## Step 1: Mount Google Drive and Import Functions

The following cell connects the notebook to Google Drive, imports the necessary functions from outside models, and sets some options.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, Seq2SeqTrainer, Seq2SeqTrainingArguments
from peft import get_peft_model, LoraConfig, TaskType
from datasets import Dataset
from pathlib import Path
import pandas as pd
filepath = Path('/content/drive/MyDrive/Text Mining Project')

Mounted at /content/drive


## Step 2: Load and Prepare the Model and Data

The following cell loads a language model from Hugging Face (in this case, the small version of the FLAN-T5 model from Google), imports the dataset, and preprocesses the data for training. Here, a PEFT version of the model is used to lessen the computational workload and make the training work on this hardware.

In [ ]:
### Load the pre-trained model and tokenizer:

model_name = 'google/flan-t5-base'
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
config = LoraConfig(
    lora_dropout=0.05,
    task_type = "SEQ_2_SEQ_LM"
)
model = get_peft_model(model, config)
model.gradient_checkpointing_enable()
model.gradient_checkpointing_enable({"use_reentrant": False})
model.config.use_cache = False
model.enable_input_require_grads()
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast = True)


### Load, prepare, and tokenize the documents for training

train_df = pd.read_csv(filepath / 'train_runs_docs.csv').iloc[:,1:]
test_df = pd.read_csv(filepath / 'test_runs_docs.csv').iloc[:,1:]
train_info = Dataset.from_pandas(train_df)
test_info = Dataset.from_pandas(test_df)

def preprocess(docs):
  """
  A helper function to preprocess/tokenize the documents and
  responses for a set of game prompts and run values

  Inputs:
  docs - a Dataset object containing game information

  Outputs:
  inputs - a Dataset object formatted for use with a Trainer object
  """

  # Tokenize inputs
  inputs = tokenizer(docs['document'], padding = 'max_length',
                     max_length = 810, truncation = True)

  # Tokenize targets
  targets = tokenizer([str(i) for i in docs['runs']],
                      padding = 'max_length', max_length = 5,
                      truncation = True)

  # Put tokenized targets in inputs Dataset
  inputs['labels'] = targets['input_ids']

  return inputs

train_data = train_info.map(preprocess, batched = True)
test_data = test_info.map(preprocess, batched = True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:72: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

Map:   0%|          | 0/5221 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

## Step 3: Run the Training

The following cell runs the training and saves the resulting fine-tuned model.

In [ ]:
### Set the training arguments:

training_args = Seq2SeqTrainingArguments(
    output_dir = "./baseball-flan-t5",
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    logging_steps = 100,
    num_train_epochs = 3,
)


### Build and run the trainer:

trainer = Seq2SeqTrainer(
    model = model,
    args = training_args,
    train_dataset = train_data,
    eval_dataset = test_data,
)

trainer.train()


### Save the fine-tuned model:

model.save_pretrained(filepath / 'score-flan-t5')
tokenizer.save_pretrained(filepath / 'score-flan-t5')

Step,Training Loss
100,15.123400
200,11.288600
300,8.851400
400,5.115200
500,2.661700
600,2.348600
700,2.276100
800,2.172300
900,2.118900
1000,2.011900


('/content/drive/MyDrive/Text Mining Project/score-flan-t5/tokenizer_config.json',
 '/content/drive/MyDrive/Text Mining Project/score-flan-t5/special_tokens_map.json',
 '/content/drive/MyDrive/Text Mining Project/score-flan-t5/tokenizer.json')